## 1. Match Data Processing

In this section, we are going to walk through the process of how we aggregate the match data to reduce the redundancy.

In [1]:
# import library
import os
import pandas as pd

In [2]:
# change the read path to dataset document stored location
!pwd
# os.chdir('Data/')

/Users/doggo/Documents/GitHub/Overwatch-League-Analysis


In [3]:
# import dataset
match_table = pd.read_csv('Data/match_map_stats.csv')
match_table.head()

round_start_time       round_end_time                       stage  \
0  2018-01-11 00:12:07  2018-01-11 00:20:07  Overwatch League - Stage 1   
1  2018-01-11 00:22:05  2018-01-11 00:27:59  Overwatch League - Stage 1   
2  2018-01-11 00:34:39  2018-01-11 00:38:29  Overwatch League - Stage 1   
3  2018-01-11 00:40:27  2018-01-11 00:44:41  Overwatch League - Stage 1   
4  2018-01-11 00:46:09  2018-01-11 00:49:48  Overwatch League - Stage 1   

   match_id  game_number         match_winner           map_winner  \
0     10223            1  Los Angeles Valiant  Los Angeles Valiant   
1     10223            1  Los Angeles Valiant  Los Angeles Valiant   
2     10223            2  Los Angeles Valiant  Los Angeles Valiant   
3     10223            2  Los Angeles Valiant  Los Angeles Valiant   
4     10223            2  Los Angeles Valiant  Los Angeles Valiant   

             map_loser          map_name  map_round  ...        team_one_name  \
0  San Francisco Shock            Dorado          1  ...  Los Angeles Valiant   
1  San Francisco Shock            Dorado          2  ...  Los Angeles Valiant   
2  San Francisco Shock  Temple of Anubis          1  ...  Los Angeles Valiant   
3  San Francisco Shock  Temple of Anubis          2  ...  Los Angeles Valiant   
4  San Francisco Shock  Temple of Anubis          3  ...  Los Angeles Valiant   

         team_two_name attacker_payload_distance defender_payload_distance  \
0  San Francisco Shock                 75.615051                  0.000000   
1  San Francisco Shock                 75.649597                 75.615051   
2  San Francisco Shock                  0.000000                  0.000000   
3  San Francisco Shock                  0.000000                  0.000000   
4  San Francisco Shock                  0.000000                  0.000000   

  attacker_time_banked defender_time_banked attacker_control_perecent  \
0             0.000000           240.000000                       NaN   
1           125.750572             0.000000                       NaN   
2           250.492004           240.000000                       NaN   
3           225.789032           250.492004                       NaN   
4            36.396057           250.492004                       NaN   

   defender_control_perecent  attacker_round_end_score  \
0                        NaN                         2   
1                        NaN                         3   
2                        NaN                         2   
3                        NaN                         2   
4                        NaN                         4   

   defender_round_end_score  
0                         0  
1                         2  
2                         0  
3                         2  
4                         2  

[5 rows x 25 columns]

In [4]:
# build function with one parameter which is the dataset being import above
def mt_func(match_table):
    # filter the dataset by column 'map_round' to only get the first round records
    map_round_1 = match_table[match_table['map_round'] == 1]
    # select the needed columns from the filtered dataset
    raw_data = map_round_1[['round_end_time','match_id', 'map_winner', 'map_loser', 'map_name', 'winning_team_final_map_score', 'losing_team_final_map_score']]
    
    # get only the year of the 'round_end_time' 
    for i in range(len(map_round_1)):
         raw_data.iloc[i,0] = int(raw_data.iloc[i,0][:4])
    
    # add column winning_team_win_rate: winning_team_final_map_score / (winning_team_final_map_score + losing_team_final_map_score)
    raw_data.loc[:, 'winning_team_win_rate'] = raw_data.iloc[:, 5] / (raw_data.iloc[:, 5] + raw_data.iloc[:, 6])
    # add column losing_team_win_rate: 1-winning_team_win_rate
    raw_data.loc[:, 'losing_team_win_rate'] = 1 - raw_data['winning_team_win_rate'] 
    
    # reorder columns 
    ordered_columns = ['round_end_time', 'match_id', 'map_name', 'map_winner', 'winning_team_final_map_score', 'winning_team_win_rate', 'map_loser', 'losing_team_final_map_score', 'losing_team_win_rate']
    result = raw_data[ordered_columns]
    
    # change one the first column name 'round_end_time' to 'match_year'
    result.rename(columns = {raw_data.columns[0]: 'match_year'}, inplace = True)
    
    return result

In [6]:
# test case
match_overall_results = mt_func(match_table)

/var/folders/jm/1f92nb4x3jsfjbh5m8xlqbw80000gn/T/ipykernel_10211/1997147631.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data.loc[:, 'winning_team_win_rate'] = raw_data.iloc[:, 5] / (raw_data.iloc[:, 5] + raw_data.iloc[:, 6])
/var/folders/jm/1f92nb4x3jsfjbh5m8xlqbw80000gn/T/ipykernel_10211/1997147631.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data.loc[:, 'losing_team_win_rate'] = 1 - raw_data['winning_team_win_rate']


In [8]:
match_overall_results.to_csv(r'Data/annual_match_result_table.csv', index=False)